# Syllable Composition Engine

> Keyman-inspired syllable composition for natural Geez input

In [ ]:
#| default_exp composition

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from typing import Dict, List, Optional, Tuple, Set
from dataclasses import dataclass
from enum import Enum
import time
from gzkeyboard.core import CharacterStore, VowelOrder

## Composition Strategy

Different strategies for handling character composition:

- **EAGER**: Output characters as soon as possible
- **LAZY**: Wait for complete input before outputting
- **PREDICTIVE**: Show predictions while typing
- **ADAPTIVE**: Adapt to user typing patterns

In [ ]:
#| export
class CompositionStrategy(Enum):
    """Different composition strategies."""
    EAGER = "eager"           # Output as soon as possible
    LAZY = "lazy"             # Wait for complete input
    PREDICTIVE = "predictive" # Show predictions
    ADAPTIVE = "adaptive"     # Adapt to user behavior

## Composition Results

The composition engine returns detailed results for each keystroke, including confidence levels and suggestions.

In [ ]:
#| export
@dataclass
class CompositionResult:
    """Result of a composition operation."""
    output           : str = ""          # Character to output
    consume_input    : int = 0           # Number of input chars to consume
    buffer_remaining : str = ""          # What stays in buffer
    confidence       : float = 1.0       # Confidence in this result (0-1)
    suggestions      : List[str] = None  # Alternative suggestions
    
    def __post_init__(self):
        if self.suggestions is None:
            self.suggestions = []

In [ ]:
# Test composition result
result = CompositionResult(output="ሃ", consume_input=2, confidence=0.9)
test_eq(result.output, "ሃ")
test_eq(result.consume_input, 2)
test_eq(result.confidence, 0.9)
test_eq(len(result.suggestions), 0)  # Should be empty list by default

## Pattern Matcher

The pattern matcher efficiently finds the best matching consonant-vowel combinations from the input buffer.

In [ ]:
#| export
class PatternMatcher:
    """Matches input patterns to character families and vowels."""
    
    def __init__(self, character_store: CharacterStore):
        self.store = character_store
        self._build_pattern_tree()
    
    def _build_pattern_tree(self):
        """Build efficient pattern matching tree."""
        self.patterns = {}
        
        # Build all possible consonant patterns
        for base_key in self.store.get_all_base_keys():
            # Add base consonant patterns
            self.patterns[base_key] = {'type': 'consonant', 'base': base_key}
            
            # Add consonant + vowel patterns
            for vowel in VowelOrder:
                pattern = base_key + vowel.latin_key
                self.patterns[pattern] = {
                    'type': 'syllable',
                    'base': base_key,
                    'vowel': vowel.latin_key,
                    'vowel_order': vowel
                }
    
    def find_best_match(self, input_text: str) -> Optional[Tuple[str, dict, int]]:
        """Find the best match for input text."""
        # Try all possible substring lengths, longest first
        for length in range(len(input_text), 0, -1):
            substring = input_text[:length]
            if substring in self.patterns:
                return (substring, self.patterns[substring], length)
        return None
    
    def has_potential_match(self, input_text: str) -> bool:
        """Check if input could potentially match longer patterns."""
        for pattern in self.patterns:
            if pattern.startswith(input_text) and len(pattern) > len(input_text):
                return True
        return False
    
    def get_completions(self, input_text: str) -> List[str]:
        """Get possible completions for partial input."""
        completions = []
        for pattern in self.patterns:
            if pattern.startswith(input_text) and len(pattern) > len(input_text):
                remaining = pattern[len(input_text):]
                completions.append(remaining)
        return completions

In [ ]:
# Test pattern matcher
store = CharacterStore()
matcher = PatternMatcher(store)

# Test exact matches
match = matcher.find_best_match('ha')
test(match is not None)
test_eq(match[0], 'ha')
test_eq(match[1]['type'], 'syllable')
test_eq(match[1]['base'], 'h')
test_eq(match[1]['vowel'], 'a')

# Test potential matches
test(matcher.has_potential_match('h'))
test(not matcher.has_potential_match('xyz'))

# Test completions
completions = matcher.get_completions('h')
test('a' in completions)
test('e' in completions)

print(f"✓ Pattern matcher tests passed")
print(f"  Found {len(matcher.patterns)} total patterns")
print(f"  Sample completions for 'h': {completions[:5]}")

## Syllable Composer

The main composition engine that handles input buffering and character generation with adaptive timing.

In [ ]:
#| export
class SyllableComposer:
    """Main syllable composition engine."""
    
    def __init__(self, 
                 character_store: CharacterStore,
                 strategy: CompositionStrategy = CompositionStrategy.ADAPTIVE,
                 timeout: float = 0.5):
        self.store = character_store
        self.matcher = PatternMatcher(character_store)
        self.strategy = strategy
        self.timeout = timeout
        
        # Input state
        self.buffer = ""
        self.last_input_time = 0.0
        
        # Statistics for adaptive behavior
        self.stats = {
            'total_inputs': 0,
            'successful_compositions': 0,
            'timeout_events': 0,
            'average_input_interval': 0.5
        }
    
    def process_input(self, key: str) -> CompositionResult:
        """Process a single keystroke and return composition result."""
        current_time = time.time()
        
        # Update statistics
        if self.last_input_time > 0:
            interval = current_time - self.last_input_time
            self._update_typing_stats(interval)
        
        self.last_input_time = current_time
        self.stats['total_inputs'] += 1
        
        # Check for timeout
        if self._is_timeout() and self.buffer:
            result = self._handle_timeout()
            self.buffer = key.lower()  # Start new sequence
            return result
        
        # Add new input to buffer
        self.buffer += key.lower()
        
        # Find matches and compose
        return self._compose_from_buffer()
    
    def _is_timeout(self) -> bool:
        """Check if current input has timed out."""
        if not self.buffer or self.last_input_time == 0:
            return False
        
        current_time = time.time()
        effective_timeout = self._calculate_adaptive_timeout()
        
        return (current_time - self.last_input_time) > effective_timeout
    
    def _calculate_adaptive_timeout(self) -> float:
        """Calculate timeout based on strategy and typing speed."""
        if self.strategy == CompositionStrategy.ADAPTIVE:
            # Adapt timeout to typing speed
            avg_interval = self.stats['average_input_interval']
            if avg_interval > 0:
                # Use 2x average interval as timeout, bounded between 0.2 and 2.0 seconds
                adaptive_timeout = min(2.0, max(0.2, avg_interval * 2))
                return adaptive_timeout
        
        return self.timeout
    
    def _update_typing_stats(self, interval: float):
        """Update typing speed statistics."""
        # Exponential moving average
        alpha = 0.1
        self.stats['average_input_interval'] = (
            alpha * interval + (1 - alpha) * self.stats['average_input_interval']
        )
    
    def _handle_timeout(self) -> CompositionResult:
        """Handle timeout event - output best match from buffer."""
        self.stats['timeout_events'] += 1
        
        match = self.matcher.find_best_match(self.buffer)
        if match:
            pattern, match_info, length = match
            output = self._generate_output(match_info)
            if output:
                remaining = self.buffer[length:]
                return CompositionResult(
                    output=output,
                    consume_input=length,
                    buffer_remaining=remaining,
                    confidence=0.8  # Lower confidence for timeout
                )
        
        # No match found - clear buffer
        return CompositionResult(
            output="",
            consume_input=len(self.buffer),
            buffer_remaining="",
            confidence=0.0
        )
    
    def _compose_from_buffer(self) -> CompositionResult:
        """Compose character from current buffer state."""
        match = self.matcher.find_best_match(self.buffer)
        has_potential = self.matcher.has_potential_match(self.buffer)
        
        if match:
            pattern, match_info, length = match
            
            # Check composition strategy
            if self.strategy == CompositionStrategy.EAGER or not has_potential:
                # Output immediately
                output = self._generate_output(match_info)
                if output:
                    self.stats['successful_compositions'] += 1
                    remaining = self.buffer[length:]
                    self.buffer = remaining
                    
                    return CompositionResult(
                        output=output,
                        consume_input=length,
                        buffer_remaining=remaining,
                        confidence=1.0,
                        suggestions=self._get_suggestions()
                    )
            
            elif self.strategy == CompositionStrategy.PREDICTIVE:
                # Show prediction but don't output yet
                completions = self.matcher.get_completions(self.buffer)
                suggestions = [self.buffer + c for c in completions[:3]]
                
                return CompositionResult(
                    output="",
                    consume_input=0,
                    buffer_remaining=self.buffer,
                    confidence=0.7,
                    suggestions=suggestions
                )
        
        # No match yet, wait for more input
        if has_potential:
            return CompositionResult(
                output="",
                consume_input=0,
                buffer_remaining=self.buffer,
                confidence=0.5,
                suggestions=self._get_suggestions()
            )
        
        # No matches and no potential - invalid input
        invalid_char = self.buffer[0] if self.buffer else ""
        self.buffer = self.buffer[1:] if len(self.buffer) > 1 else ""
        
        return CompositionResult(
            output=invalid_char,  # Pass through invalid character
            consume_input=1,
            buffer_remaining=self.buffer,
            confidence=0.0
        )
    
    def _generate_output(self, match_info: dict) -> str:
        """Generate output character from match information."""
        if match_info['type'] == 'syllable':
            return self.store.get_character(match_info['base'], match_info['vowel'])
        elif match_info['type'] == 'consonant':
            # Base consonant - use default vowel (ግዕዝ)
            return self.store.get_character(match_info['base'], '')
        return ""
    
    def _get_suggestions(self) -> List[str]:
        """Get suggestions for current buffer state."""
        completions = self.matcher.get_completions(self.buffer)
        return [self.buffer + c for c in completions[:3]]
    
    def reset(self):
        """Reset composer state."""
        self.buffer = ""
        self.last_input_time = 0.0
    
    def get_stats(self) -> dict:
        """Get composition statistics."""
        success_rate = 0.0
        if self.stats['total_inputs'] > 0:
            success_rate = self.stats['successful_compositions'] / self.stats['total_inputs']
        
        typing_speed = 0.0
        if self.stats['average_input_interval'] > 0:
            typing_speed = 1.0 / self.stats['average_input_interval']
        
        return {
            **self.stats,
            'success_rate': success_rate,
            'typing_speed_cps': typing_speed,
            'current_buffer': self.buffer,
            'adaptive_timeout': self._calculate_adaptive_timeout()
        }

## Testing the Composition Engine

Let's test our new composition engine with various scenarios.

In [ ]:
# Test syllable composer
store = CharacterStore()
composer = SyllableComposer(store, strategy=CompositionStrategy.EAGER)

# Test basic composition
result = composer.process_input('h')
test_eq(result.output, '')  # No output yet, waiting for vowel
test_eq(composer.buffer, 'h')

result = composer.process_input('a')
test_eq(result.output, 'ሃ')  # Should output ሃ
test_eq(result.consume_input, 2)
test_eq(composer.buffer, '')  # Buffer should be cleared

print(f"✓ Basic composition test passed")
print(f"  'ha' → '{result.output}' with confidence {result.confidence}")

In [ ]:
# Test multiple character sequences
test_sequences = [
    ('s', 'e', 'ሴ'),  # Should produce ሴ
    ('l', 'o', 'ሎ'),  # Should produce ሎ
    ('m', 'a', 'ማ'),  # Should produce ማ
]

print("\nTesting multiple sequences:")
for char1, char2, expected in test_sequences:
    composer.reset()
    composer.process_input(char1)
    result = composer.process_input(char2)
    test_eq(result.output, expected)
    print(f"  '{char1}{char2}' → '{result.output}' ✓")

In [ ]:
# Test different strategies
strategies = [CompositionStrategy.EAGER, CompositionStrategy.LAZY, CompositionStrategy.PREDICTIVE]

print("\nTesting different strategies:")
for strategy in strategies:
    composer = SyllableComposer(store, strategy=strategy)
    composer.process_input('h')
    result = composer.process_input('e')
    print(f"  {strategy.value:12}: 'he' → '{result.output}' (confidence: {result.confidence:.2f})")

## Interactive Testing Functions

Create helper functions for interactive testing and demonstration.

In [ ]:
#| export
def test_composition_interactive(composer: SyllableComposer, input_text: str, show_details: bool = True):
    """Test composition with detailed step-by-step output."""
    if show_details:
        print(f"\nTesting composition: '{input_text}' (strategy: {composer.strategy.value})")
        print("=" * 60)
    
    composer.reset()
    output = ""
    
    for i, char in enumerate(input_text):
        result = composer.process_input(char)
        
        if show_details:
            status = f"Input: '{char}' → Output: '{result.output}'"
            status += f" | Buffer: '{composer.buffer}' | Confidence: {result.confidence:.2f}"
            print(f"  Step {i+1}: {status}")
            
            if result.suggestions:
                print(f"    Suggestions: {result.suggestions}")
        
        output += result.output
    
    if show_details:
        stats = composer.get_stats()
        print(f"\nFinal output: '{output}'")
        print(f"Statistics: Success rate: {stats['success_rate']:.2f}, "
              f"Typing speed: {stats['typing_speed_cps']:.1f} cps, "
              f"Adaptive timeout: {stats['adaptive_timeout']:.3f}s")
    
    return output

def compare_strategies(input_text: str):
    """Compare different composition strategies."""
    store = CharacterStore()
    strategies = [CompositionStrategy.EAGER, CompositionStrategy.LAZY, 
                  CompositionStrategy.PREDICTIVE, CompositionStrategy.ADAPTIVE]
    
    print(f"\nComparing strategies for input: '{input_text}'")
    print("=" * 50)
    
    for strategy in strategies:
        composer = SyllableComposer(store, strategy=strategy)
        output = test_composition_interactive(composer, input_text, show_details=False)
        stats = composer.get_stats()
        print(f"{strategy.value:12}: '{output}' (success rate: {stats['success_rate']:.2f})")

def demo_adaptive_behavior(texts: List[str]):
    """Demonstrate adaptive behavior over multiple inputs."""
    store = CharacterStore()
    composer = SyllableComposer(store, strategy=CompositionStrategy.ADAPTIVE)
    
    print("\nDemonstrating adaptive behavior:")
    print("=" * 40)
    
    for i, text in enumerate(texts):
        output = test_composition_interactive(composer, text, show_details=False)
        stats = composer.get_stats()
        print(f"Text {i+1}: '{text}' → '{output}' "
              f"(timeout: {stats['adaptive_timeout']:.3f}s, speed: {stats['typing_speed_cps']:.1f} cps)")
        
        # Simulate typing delay between words
        time.sleep(0.1)

In [ ]:
# Interactive testing examples
store = CharacterStore()
composer = SyllableComposer(store, strategy=CompositionStrategy.ADAPTIVE)

# Test with common words
test_words = ['hello', 'selam', 'geez']

for word in test_words:
    output = test_composition_interactive(composer, word)

In [ ]:
# Compare strategies
compare_strategies('hello')
compare_strategies('selam')

In [ ]:
# Demonstrate adaptive behavior
demo_texts = ['ha', 'he', 'hello', 'selam', 'tigrinya']
demo_adaptive_behavior(demo_texts)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()